In [85]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import log, dot
import pandas as pd
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Arrhythmia

Adapted from [Codes for Unit 7: arrhythmia.odc](https://www2.isye.gatech.edu/isye6420/supporting.html).

Associated lecture video: [Unit 7 Lesson 15](https://www.youtube.com/watch?v=999v5stS8jw&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=77).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/arrhythmia.csv).

Patients who undergo Coronary Artery Bypass Graft Surgery (CABG) have an approximate 19-40% chance of developing atrial fibrillation (AF). AF can lead to blood clots forming causing greater in-hospital mortality, strokes, and longer hospital stays. While this can be prevented with drugs, it is very expensive and sometimes dangerous if not warranted. Ideally, several risk factors which would indicate an increased risk of developing AF in this population could save lives and money by indicating which patients need pharmacological intervention. Researchers began collecting data from CABG patients during their hospital stay such as demographics like age and sex, as well as heart rate, cholesterol, operation time, etc.. Then, the researchers recorded which patients developed AF during their hospital stay. Researchers now want to find those pieces of data which indicate high risk of AF. In the past, indicators like age, hypertension, and body surface area (BSA) have been good indicators, though these alone have not produced a satisfactory solution.

Fibrillation occurs when the heart muscle begins a quivering motion instead of a normal, healthy pumping rhythm. Fibrillation can affect the atrium (atrial fibrillation) or the ventricle (ventricular  fibrillation); ventricular fibrillation is imminently life threatening.

Atrial fibrillation is the quivering, chaotic motion in the upper chambers of the heart, known as the atria. Atrial fibrillation is often due to serious underlying medical conditions, and should be evaluated by a physician. It is not typically a medical emergency.

Ventricular fibrillation occurs in the ventricles (lower chambers) of the heart; it is always a medical emergency. If left untreated, ventricular fibrillation (VF, or V-fib) can lead to death within minutes. When a heart goes into V-fib, effective pumping of the blood stops. V-fib is considered a form of cardiac arrest, and an individual suffering from it will not survive unless cardiopulmonary resuscitation (CPR) and defibrillation are provided immediately.

DATA Arrhythmia
- Y = Fibrillation
- X1 = Age
- X2 = Aortic Cross Clamp Time
- X3 = Cardiopulmonary Bypass Time:
    - Bypass of the heart and lungs as, for example, in open heart surgery. Blood returning to the heart is diverted through a heart-lung machine (a pump-oxygenator) before returning it to the arterial circulation. The machine does the work both of the heart (pump blood) and the lungs (supply oxygen to red blood cells).
- X4 = ICU Time	(Intensive Care Unit)
- X5 = Avg Heart Rate	
- X6 = Left Ventricle Ejection Fraction
- X7 = Hypertension
- X8 = Gender [1 -Female; 0-Male]
- X9 = Diabetis
- X10 = Previous MI


----------------------------

Just throwing this up real quick because it might be useful for homework 5. Note this is using PyMC 4.0 beta 3. I didn't add the residual calculation yet because I don't think it's required for this homework.

In [24]:
%watermark --iversions

numpy : 1.22.3
pymc  : 4.0.0b3
pandas: 1.4.1
arviz : 0.11.4



In [79]:
data_df = pd.read_csv("./data/arrhythmia.csv")
data_df.info()
X = data_df.iloc[:, 1:].values
# add intercept column to X
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = data_df["Fibrillation"].values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Fibrillation                   81 non-null     float64
 1   Age                            81 non-null     float64
 2   AorticCrossClampTime           81 non-null     float64
 3   CardiopulmonaryBypassTime      81 non-null     float64
 4   ICUTime                        81 non-null     float64
 5   AvgHeartRate                   81 non-null     float64
 6   LeftVentricleEjectionFraction  81 non-null     float64
 7   Hypertension                   81 non-null     float64
 8   Gender                         81 non-null     float64
 9   Diabetes                       81 non-null     float64
 10  PreviousMI                     81 non-null     float64
dtypes: float64(11)
memory usage: 7.1 KB


In [88]:
with pm.Model() as m:
    X_data = pm.Data("X_data", X_aug)
    y_data = pm.Data("y_data", y)

    betas = pm.Normal("beta", mu=0, tau=0.001, shape=X.shape[1] + 1)

    p = dot(X_data, betas)

    lik = pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(
        10000,
        chains=4,
        tune=500,
        cores=4,
        random_seed=1,
    )

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/data.py:647: FutureWarning: The `mutable` kwarg was not specified. Currently it defaults to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default will change to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Set `pm.Data(..., mutable=False/True)`, or use `pm.ConstantData`/`pm.MutableData`.
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 500 tune and 10_000 draw iterations (2_000 + 40_000 draws total) took 37 seconds.
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/backends/arviz.py:57: UserWarning: Could

In [90]:
az.summary(trace, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-13.114,4.920,-22.873,-3.672,0.039,0.027,16349.0,22260.0,1.0
beta[1],0.189,0.050,0.098,0.292,0.000,0.000,21662.0,22611.0,1.0
beta[2],0.033,0.026,-0.019,0.083,0.000,0.000,18511.0,23081.0,1.0
beta[3],-0.024,0.016,-0.056,0.007,0.000,0.000,19296.0,23479.0,1.0
beta[4],-0.154,0.096,-0.335,0.044,0.001,0.000,36226.0,25276.0,1.0
beta[5],0.006,0.032,-0.057,0.068,0.000,0.000,25412.0,27080.0,1.0
beta[6],0.025,0.028,-0.030,0.081,0.000,0.000,25144.0,27201.0,1.0
beta[7],-0.662,0.674,-2.017,0.627,0.004,0.003,36451.0,27743.0,1.0
beta[8],-0.328,0.693,-1.681,1.041,0.003,0.003,40394.0,27906.0,1.0
beta[9],1.323,0.695,-0.006,2.720,0.004,0.003,27349.0,26665.0,1.0
